## 命令行推理

### For Windows

In [ ]:
!chcp 65001

### For Linux

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

## API Client

需要在终端开启API Server

> 音频用本地路径

> 文本可以直接用路径，也可以用内容

In [ ]:
!python -m tools.post_api \
    --text "Hello everyone, I am an open-source text-to-speech model developed by Fish Audio." \
    --reference_audio "D:\PythonProject\原神语音中文\胡桃\vo_hutao_draw_appear.wav" \
    --reference_text "D:\PythonProject\原神语音中文\胡桃\vo_hutao_draw_appear.lab" \
    --streaming True

## For Test

### 0. 下载模型

In [ ]:
!set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com
!huggingface-cli download fishaudio/fish-speech-1.2 --local-dir checkpoints/fish-speech-1.2/

### 1. 从语音生成 prompt:
>  如果你打算让模型随机选择音色, 你可以跳过这一步.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## 在此输入你的语音路径:
src_audio = r"D:\PythonProject\原神语音中文\胡桃\vo_hutao_draw_appear.wav"

!python tools/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.2/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. 从文本生成语义 token:
> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 --compile 来融合 cuda 内核以实现更快的推理

In [ ]:
!python tools/llama/generate.py \
    --text "人间灯火倒映湖中，她的渴望让静水泛起涟漪。若代价只是孤独，那就让这份愿望肆意流淌。流入她所注视的世间，也流入她如湖水般澄澈的目光。" \
    --prompt-text "唷，找本堂主有何贵干呀？嗯？你不知道吗，往生堂第七十七代堂主就是胡桃我啦！嘶，不过瞧你的模样，容光焕发，身体健康，嗯…想必是为了工作以外的事来找我，对吧？" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2" \
    --num-samples 2
    # --compile

### 3. 从语义 token 生成人声:

In [ ]:
!python tools/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)